In [ ]:
import os,sys
from collections import defaultdict
import json
import numpy as np
import pickle
from skimage import io
import matplotlib
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm
from scipy.ndimage.morphology import distance_transform_edt
%load_ext autoreload
%autoreload 2

In [ ]:
PATH = '/home/eddyod/programming/pipeline_utility'

sys.path.append(PATH)
from utilities.FileLocationManager import DATA_PATH, ROOT_DIR

from utilities.SqlController import SqlController
from utilities.atlas.imported_atlas_utilities import load_original_volume_all_known_structures_v3, get_centroid_3d, \
    load_alignment_results_v3, transform_points, average_location, \
    convert_to_original_name, name_unsided_to_color, paired_structures, \
    convert_to_left_name, convert_to_right_name, load_original_volume_v2, save_alignment_results_v3, \
    convert_transform_forms, transform_volume_v4, volume_to_polydata, singular_structures, \
    MESH_DIR, average_shape, convert_to_surround_name, mirror_volume_v2, save_original_volume, \
    save_mesh_stl, get_surround_volume_v2, transform_volume_v4, high_contrast_colors, \
    plot_centroid_means_and_covars_3d, all_known_structures_sided, load_data, \
    get_instance_mesh_filepath, images_to_volume_v2, find_contour_points, load_cropbox_v2, \
    load_mean_shape, \
    display_volume_sections, get_structure_mean_positions_filepath
from utilities.atlas.atlas_aligner import Aligner
from utilities.utilities_alignment import convert_resolution_string_to_um
atlas_name = 'atlasV8'
ATLAS_PATH = os.path.join(DATA_PATH, 'atlas_data', atlas_name)

In [ ]:
fixed_brain_name = 'MD589'
sqlController = SqlController(fixed_brain_name)
structures = sqlController.get_structures_list()
structures.remove("R")
moving_brain_names = ['MD585', 'MD594']
resolution = '10.0um'
resolution_um = 10.0
structure_centroids_all_brains_um_wrt_fixed = []
fixed_brain_spec = {'name': fixed_brain_name, 'vol_type': 'annotationAsScore', 'resolution': resolution}

In [ ]:
fixed_brain = load_original_volume_all_known_structures_v3(stack_spec=fixed_brain_spec, structures=structures, 
                                                           in_bbox_wrt='wholebrain')
fixed_brain_structure_centroids = get_centroid_3d(fixed_brain)
fixed_brain_structure_centroids_um = {s: c * resolution_um for s, c in fixed_brain_structure_centroids.items()}
structure_centroids_all_brains_um_wrt_fixed.append(fixed_brain_structure_centroids_um)

## Compute instance centroids

In [ ]:
for brain_m in moving_brain_names:
    moving_brain_spec = {'name': brain_m, 'vol_type': 'annotationAsScore', 'resolution': resolution}
    print('Brain', moving_brain_spec)
    moving_brain = load_original_volume_all_known_structures_v3(stack_spec=moving_brain_spec, 
                                                                structures=structures, in_bbox_wrt='wholebrain')
    alignment_spec = dict(stack_m=moving_brain_spec, stack_f=fixed_brain_spec, warp_setting=109)
    moving_brain_structure_centroids_input_resol = get_centroid_3d(moving_brain)
    # Load registration.
    # Alignment results fp: os.path.join(reg_root_dir, alignment_spec['stack_m']['name'], warp_basename, warp_basename + '_' + what + '.' + ext)
    transform_parameters_moving_brain_to_fixed_brain = load_alignment_results_v3(alignment_spec=alignment_spec, what='parameters')
    # Transform moving brains into alignment with the fixed brain.
    transformed_moving_brain_structure_centroids_input_resol_wrt_fixed = \
    dict(list(zip(list(moving_brain_structure_centroids_input_resol.keys()),
                  transform_points(pts=list(moving_brain_structure_centroids_input_resol.values()),
                                   transform=transform_parameters_moving_brain_to_fixed_brain))))

    transformed_moving_brain_structure_centroids_um_wrt_fixed = \
        {s: c * resolution_um for s, c in
        list(transformed_moving_brain_structure_centroids_input_resol_wrt_fixed.items())}

    structure_centroids_all_brains_um_wrt_fixed.append(transformed_moving_brain_structure_centroids_um_wrt_fixed)


In [ ]:
structure_centroids_all_brains_um_grouped_by_structure_wrt_fixed = defaultdict(list)
for sc in structure_centroids_all_brains_um_wrt_fixed:
    for k, c in sc.items():
        structure_centroids_all_brains_um_grouped_by_structure_wrt_fixed[k].append(c)
structure_centroids_all_brains_um_grouped_by_structure_wrt_fixed.default_factory = None

## Compute standard centroids

In [ ]:
nominal_centroids_wrt_canonicalAtlasSpace_um, \
instance_centroids_wrt_canonicalAtlasSpace_um, \
canonical_center_wrt_fixed_um, \
canonical_normal, \
transform_matrix_to_canonicalAtlasSpace_um = \
average_location(structure_centroids_all_brains_um_grouped_by_structure_wrt_fixed)

In [ ]:
colors = {name_s: np.array(name_unsided_to_color[convert_to_original_name(name_s)])/255.
                                        for name_s in instance_centroids_wrt_canonicalAtlasSpace_um.keys()}

In [ ]:
plot_centroid_means_and_covars_3d(instance_centroids=instance_centroids_wrt_canonicalAtlasSpace_um,
                                 nominal_locations=nominal_centroids_wrt_canonicalAtlasSpace_um,
                                 canonical_centroid=(0,0,0),
                                  show_canonical_centroid=True,
                                  canonical_normal=[0,0,1],
                                 colors=colors,
                                 xlim=[-3000, 3000],
                                 ylim=[-3000, 3000],
                                 zlim=[-3000, 3000],
                                 xlabel='Rostral-caudal ($\mu$m)',
                                 ylabel='',
                                 zlabel='Medial-lateral ($\mu$m)',
                                 title='Centroid means and covariances (3 brains)')

In [ ]:
filepath = os.path.join(ATLAS_PATH, '1um_meanPositions.pkl')
with open(filepath, 'wb') as f:
    pickle.dump(nominal_centroids_wrt_canonicalAtlasSpace_um, f)

In [ ]:
filepath = os.path.join(ATLAS_PATH, 'canonicalCentroid_wrt_fixedWholebrain.txt')
np.savetxt(filepath, canonical_center_wrt_fixed_um)

In [ ]:
reg_root_dir=os.path.join(ATLAS_PATH, 'mean_shapes', 'instance_registration')

In [ ]:
# Note that all shapes have voxel resolution matching input resolution (10.0 micron).
for structure in tqdm(structures):
    # for structure in all_known_structures:
    # Load instance volumes.
    instance_volumes = []
    instance_source = []

    for brain_name in [fixed_brain_name] + moving_brain_names:
        brain_spec = {'name': brain_name, 'vol_type': 'annotationAsScore', 'resolution': resolution}
       
        if '_L' in structure:
            left_instance_vol, _ = load_original_volume_v2(stack_spec=brain_spec,
                                                           structure=structure,
                                                           return_origin_instead_of_bbox=True,
                                                           crop_to_minimal=True)
            instance_volumes.append(left_instance_vol[..., ::-1])  # if left, mirror
            instance_source.append((brain_name, 'L'))
        
        else:
            right_instance_vol, _ = load_original_volume_v2(stack_spec=brain_spec,
                                                            structure=structure,
                                                            return_origin_instead_of_bbox=True,
                                                            crop_to_minimal=True)
            instance_volumes.append(right_instance_vol)  # if right, do not mirror
            instance_source.append((brain_name, 'R'))


   # Use the first instance as registration target.
    # Register every other instance to the first instance.
    template_instance_volume = instance_volumes[0]
    template_instance_centroid_wrt_templateOrigin = get_centroid_3d(template_instance_volume).astype(np.int16)
    template_instance_wrt_templateCentroid = (template_instance_volume, - template_instance_centroid_wrt_templateOrigin)
    aligned_moving_instance_wrt_templateCentroid_all_instances = []

    for i in range(1, len(instance_volumes)):
        # Compute transform.
        moving_instance_volume = instance_volumes[i]
        aligner = Aligner({0: template_instance_wrt_templateCentroid},
                          {0: (moving_instance_volume, np.array((0,0,0)))},
                          labelIndexMap_m2f={0:0},
                         verbose=False)
        aligner.set_centroid(centroid_m='structure_centroid', centroid_f='structure_centroid')
        aligner.compute_gradient(smooth_first=True)
        lr = 1.
        ### max_iter_num was originally 100 and 1000
        _, _ = aligner.optimize(tf_type='rigid',
                                history_len=100,
                                max_iter_num=2 if structure in ['SC', 'IC'] else 3,
                                grad_computation_sample_number=None,
                                full_lr=np.array([lr, lr, lr, 0.1, 0.1, 0.1]),
                                terminate_thresh_trans=.01)



        reg_root_dir=os.path.join(ATLAS_PATH, 'mean_shapes', 'instance_registration')
        save_alignment_results_v3(aligner=aligner,
                              select_best='max_value',
                              alignment_spec=dict(warp_setting=108,
                                                  stack_f=dict(name='%s_instance0' % structure, vol_type='annotationAsScore'),
                                                  stack_m=dict(name='%s_instance%d' % (structure, i),
                                                               vol_type='annotationAsScore')),
                              reg_root_dir=reg_root_dir)

        # Transform instances.
        T = convert_transform_forms(aligner=aligner, out_form=(3, 4), select_best='max_value')
        aligned_moving_instance_volume, aligned_moving_instance_origin_wrt_templateCentroid = \
            transform_volume_v4(volume=(moving_instance_volume, (0, 0, 0)), transform=T,
                                return_origin_instead_of_bbox=True)
        aligned_moving_instance_wrt_templateCentroid = (
        aligned_moving_instance_volume, aligned_moving_instance_origin_wrt_templateCentroid)
        aligned_moving_instance_wrt_templateCentroid_all_instances.append(aligned_moving_instance_wrt_templateCentroid)

    # Generate meshes for each instance.
    volume_origin_list = [template_instance_wrt_templateCentroid] + aligned_moving_instance_wrt_templateCentroid_all_instances
    instance_mesh_wrt_templateCentroid_all_instances = [volume_to_polydata(volume, num_simplify_iter=3, smooth=True)
        for volume, o in volume_origin_list]

    # Save meshes.
    #for i, mesh_data in enumerate(instance_mesh_wrt_templateCentroid_all_instances):
    #    meshfile = '{}_{}_{}.stl'.format(resolution, structure, str(i))
    #    meshpath = os.path.join(ATLAS_PATH, 'aligned_instance_meshes', meshfile)
    #    #print('Save stl at {}'.format( meshpath))
    #    save_mesh_stl(mesh_data, meshpath)

    #filename = '{}_sources.pkl'.format(structure)
    #filepath = os.path.join(ATLAS_PATH, 'instance_sources', filename)
    #with open(filepath, 'wb') as f:
    #    pickle.dump(instance_source, f)

    # Compute average shape.

    if structure == 'IC' or structure == 'SC':
        # IC and SC boundaries are particularly jagged, so do a larger value smoothing.
        sigma = 5.
    else:
        sigma = 2.


    mean_shape_wrt_templateCentroid = \
        average_shape(volume_origin_list=volume_origin_list, force_symmetric=(structure in singular_structures),
                      sigma=sigma,
                      )

    # Generate meshes for mean shape.
    #mean_shape_isosurface_polydata_all_levels = {surface_level:
    #                                                 volume_to_polydata(
    #                                                     (mean_shape_wrt_templateCentroid[0] >= surface_level,
    #                                                     mean_shape_wrt_templateCentroid[1]),
    #                                                     num_simplify_iter=3, smooth=True)
    #    for surface_level in np.arange(0.1, 1.1, .1)}

    # Identify the surrouding area as additional structure.

    wall_level = .5
    surround_distance_um = 200.

    # changed to v2 to v3 Jul/27/2020 renamed without the _vX
    # volume, distance=5, wall_level=0, prob=False, return_origin_instead_of_bbox=True, padding=5
    # def get_surround_volume(volume, origin, distance=5, wall_level=0, prob=False, return_origin_instead_of_bbox=True,
    #                        padding=5):
    surround_wrt_stdShapeCentroid = \
        get_surround_volume_v2(vol=mean_shape_wrt_templateCentroid[0],
                               origin=mean_shape_wrt_templateCentroid[1],
                               distance=surround_distance_um / resolution_um,
                               wall_level=wall_level,
                               prob=True,
                               return_origin_instead_of_bbox=True,
                               padding=5)

    # Generate meshes for surrouding area.
    #surround_isosurface_polydata_all_levels = {surface_level:
    #         volume_to_polydata((surround_wrt_stdShapeCentroid[0] >= surface_level,
    #                            surround_wrt_stdShapeCentroid[1]),
    #                            num_simplify_iter=3, smooth=True)
    #     for surface_level in np.arange(0.1, 1.1, .1)}

    # Save mean shape.
    filename = f'{structure}.npy'
    filepath =  os.path.join(ATLAS_PATH, 'structure', filename)
    np.save(filepath, np.ascontiguousarray(mean_shape_wrt_templateCentroid[0]))

    filename = f'{structure}.txt'
    filepath = os.path.join(ATLAS_PATH, 'origin', filename)
    np.savetxt(filepath, mean_shape_wrt_templateCentroid[1])


    #for level in np.arange(0.1, 1.1, .1):
    #    filename = '{}_mesh_level_{}.stl'.format(structure, str(level))
    #    filepath = os.path.join(ATLAS_PATH, 'mean_shapes', filename)
    #    save_mesh_stl(mean_shape_isosurface_polydata_all_levels[level], filepath)

    surround_name = convert_to_surround_name(structure, margin=str(int(surround_distance_um)) + 'um')
    filename = '{}_volume.npy'.format(surround_name)
    filepath = os.path.join(ATLAS_PATH, 'mean_shapes', filename)
    np.save(filepath, np.ascontiguousarray(surround_wrt_stdShapeCentroid[0]))


    filename = '{}_origin_wrt_meanShapeCentroid.txt'.format(surround_name)
    filepath = os.path.join(ATLAS_PATH, 'mean_shapes', filename)
    np.savetxt(filepath, surround_wrt_stdShapeCentroid[1])

    #for level in np.arange(0.1, 1.1, .1):
    #    filename = '{}_{}.stl'.format(surround_name, str(level))
    #    filepath = os.path.join(ATLAS_PATH, 'mean_shapes', filename)
    #    save_mesh_stl(surround_isosurface_polydata_all_levels[level], filepath)


In [ ]:
template_instance_wrt_templateCentroid[1]

In [ ]:
i = 1
# Compute transform.
moving_instance_volume = instance_volumes[i]
aligner = Aligner({0: template_instance_wrt_templateCentroid}, 
                  {0: (moving_instance_volume, np.array((0,0,0)))}, 
                  labelIndexMap_m2f={0:0})
aligner.set_centroid(centroid_m='structure_centroid', centroid_f='structure_centroid')
aligner.compute_gradient(smooth_first=True)
lr = .1
_, _ = aligner.optimize(tf_type='rigid', 
                     history_len=100, 
                    max_iter_num=100 if structure in ['SC', 'IC'] else 500,
                     grad_computation_sample_number=None,
                        full_lr=np.array([lr,lr,lr,0.1,0.1,0.1]),
                       terminate_thresh_trans=.01)

In [ ]:
moving_instance_volume.shape, moving_instance_volume.dtype

In [ ]:
plt.plot(aligner.Ts);

In [ ]:
plt.plot(aligner.scores);

In [ ]:
# doesn't display much
# Transform instances.

T = convert_transform_forms(aligner=aligner, out_form=(3,4), select_best='max_value')

aligned_moving_instance_volume, aligned_moving_instance_origin_wrt_templateCentroid = \
transform_volume_v4((moving_instance_volume, (0,0,0)), transform=T,
                    return_origin_instead_of_bbox=True)

aligned_moving_instance_wrt_templateCentroid = (aligned_moving_instance_volume, aligned_moving_instance_origin_wrt_templateCentroid)        

# Generate meshes for each instance.

instance_mesh_wrt_templateCentroid_all_instances = [
volume_to_polydata((v, o), num_simplify_iter=3, smooth=True)
for v, o in 
[template_instance_wrt_templateCentroid] + [aligned_moving_instance_wrt_templateCentroid]]

# Visualize Results

# Combine standard shapes with standard centroid locations

In [ ]:
atlas_resolution = '10.0um'
atlas_resolution_um = 10.0

In [ ]:
atlas_spec = dict(name=atlas_name, vol_type='score', resolution=atlas_resolution)

In [ ]:
mean_shapes.keys()

In [ ]:
filepath = os.path.join(ATLAS_PATH, '1um_meanPositions.pkl')
nominal_centroids = pickle.load( open(filepath, "rb" ) )
nominal_centroids_10um = {s: c / atlas_resolution_um for s, c in nominal_centroids.items()}
mean_shapes = {name_u: load_mean_shape(atlas_name=atlas_name, structure=name_u, resolution=atlas_resolution) 
                    for name_u in structures}

In [ ]:
for structure in structures:

    if '_L' in structure:
        mean_shape = mirror_volume_v2(volume=mean_shapes[structure], 
                                      centroid_wrt_origin=-mean_shapes[structure][1],
                                      new_centroid=nominal_centroids_10um[structure])
    else:
        mean_shape = (mean_shapes[structure][0], 
                        mean_shapes[structure][1] + nominal_centroids_10um[structure])
        
    volume = mean_shape[0]
    origin = mean_shape[1]
    print(structure, volume.shape, origin )
    
    # save origin, this is also the important one
    filename = f'{structure}.txt'
    filepath = os.path.join(ATLAS_PATH, 'origin', filename)
    np.savetxt(filepath, origin)

    # Save volume with stated level. This is the important one
    filename = f'{structure}.npy'
    filepath = os.path.join(ATLAS_PATH, 'structure', filename)
    np.save(filepath, volume)






# Convert to vtk polydata for visualization

In [ ]:
#for name_s in all_known_structures_sided_including_surround_200um:
for name_s in structures:
    atlas_structure_wrt_canonicalAtlasSpace = \
    load_original_volume_v2(stack_spec=atlas_spec, structure=name_s, bbox_wrt='canonicalAtlasSpace')

    for surface_level in np.arange(0.1, 1.1, .1):
        mean_shape_isosurface_polydata_wrt_canonicalAtlasSpace = \
        volume_to_polydata(volume=(atlas_structure_wrt_canonicalAtlasSpace[0] >= surface_level, atlas_structure_wrt_canonicalAtlasSpace[1]), 
                     num_simplify_iter=3, smooth=True, 
                     return_vertex_face_list=False)
        path = '/home/eddyod/tmp/meshes'
        if surface_level == 0.2:
            filepath = os.path.join(path, '{}_{}.stl'.format(name_s, surface_level))
            save_mesh_stl(mean_shape_isosurface_polydata_wrt_canonicalAtlasSpace, filepath)
        #save_data(mean_shape_isosurface_polydata_wrt_canonicalAtlasSpace, 
        #          DataManager.get_mesh_filepath_v2(atlas_spec, structure=name_s, level=surface_level))

In [ ]:
mean_shape_level05isosurface_polydata_wrt_canonicalAtlasSpace_all_structures = \
{name_s: load_data(DataManager.get_mesh_filepath_v2(atlas_spec, structure=name_s, level=0.5))
for name_s in all_known_structures_sided}

In [ ]:
#launch_vtk([actor_mesh(v, wireframe=False, opacity=.5, color=name_unsided_to_color_float[convert_to_original_name(s)]) 
#            for s, v in mean_shape_level05isosurface_polydata_wrt_canonicalAtlasSpace_all_structures.iteritems()] \
#          + [actor_sphere([0,0,0])])

# Shell

Just use MD589's shell, until we find a way to average the outlines.

In [ ]:
stack_fixed = 'MD589'
MASKS = '/net/birdstore/Active_Atlas_Data/data_root/brains_info/masks/{}/aligned'.format(stack_fixed)
margin_um = 200
in_resolution_um = 0.46 * 32
margin_tb = margin_um / XY_PIXEL_DISTANCE_TB
for stack in ['MD589']:
    #sqlController = SqlController(stack)
#     contours_valid_by_z = {}
    contour_prob_maps_by_sec = {} 
    sections = sorted(os.listdir(MASKS))
    i = 1
    for sec in sections[0:10]:
        maskfile = os.path.join(MASKS, sec)
        m = io.imread(maskfile)
        contours = find_contour_points(m, sample_every=1)[1]
        contours_valid = [cnt[(cnt[:,0] >= 1) & (cnt[:,1] >= 1)] for cnt in contours]
#         z = np.mean(DataManager.convert_section_to_z(stack, sec, downsample=32, z_begin=0))
#         contours_valid_by_z[z] = contours_valid[0]
        m2 = np.zeros_like(m, np.bool)
        for cnt in contours_valid:
            m2[cnt[:,1], cnt[:,0]] = 1
        distance_to_contour = distance_transform_edt(~m2)
        contour_prob_map = np.exp(-distance_to_contour**2/(2*margin_tb)**2)
        contour_prob_map[contour_prob_map < 1e-2] = 0
        contour_prob_maps_by_sec[i] = contour_prob_map
        i += 1
        
outline_prob_volume_atlasResol, outline_prob_volume_origin_wrt_wholebrainXYcropped_atlasResol = \
images_to_volume_v2(images=contour_prob_maps_by_sec, spacing_um=20, 
                    in_resol_um = in_resolution_um,
                    out_resol_um = atlas_resolution_um)                                                                    
#outline_prob_volume_bbox (xm,xm,ym,ym) relative to cropped, (zm,zm) relative to uncropped.
# crop_box = metadata_cache['cropbox']['MD589']
alignedBrainstemCrop_cropbox_down32 = load_cropbox_v2(stack=stack_fixed, prep_id='alignedBrainstemCrop')
alignedBrainstemCrop_cropbox_atlasResol = alignedBrainstemCrop_cropbox_down32 * in_resolution_um / atlas_resolution_um
outline_prob_volume_origin_rel2fixedwholebrain_atlasResol = outline_prob_volume_origin_wrt_wholebrainXYcropped_atlasResol + (alignedBrainstemCrop_cropbox_atlasResol[0], alignedBrainstemCrop_cropbox_atlasResol[2], 0)
#display_volume_sections(outline_prob_volume, direction='z', ncols=5, cmap=plt.cm.gray, start_level=0)

In [ ]:
outline_prob_volume_origin_rel2canon_atlasResol = outline_prob_volume_origin_rel2fixedwholebrain_atlasResol - \
canonical_center_wrt_fixed_um/convert_resolution_string_to_um(resolution='10.0um')

In [ ]:
shell_vol_origin_dict_rel2canon = {'shell': (outline_prob_volume_atlasResol, outline_prob_volume_origin_rel2canon_atlasResol)}

# Side task: compute the volumes of structures

In [ ]:
atlas_resolution = '10.0um'
atlas_resolution_um = 10.0
atlas_spec = dict(name='atlasV7', vol_type='score', resolution=atlas_resolution)
volumes = load_original_volume_all_known_structures_v3(atlas_spec, 
                           structures=all_known_structures_sided, in_bbox_wrt='canonicalAtlasSpace')

volumes_mm3 = defaultdict(dict)
structures = ['SC', 'IC']
for name_u in structures:
    for level in np.arange(0, 1.1, .1):
        volumes_mm3[name_u][level] = np.count_nonzero(volumes[convert_to_left_name(name_u)][0] > level) * 10.**3 / 1e9

In [ ]:
df = pd.DataFrame(volumes_mm3)

In [ ]:
df.head()

# Load

In [ ]:
standard_shape_level05surface_polydata_wrt_canonicalAtlasSpace_all_structures = \
DataManager.load_meshes_v2(atlas_spec, levels=.5, include_surround=True, 
                           structures=all_known_structures_sided)
#                            structures=all_known_structures_sided + all_known_structures_sided_with_surround_200um)

In [ ]:
launch_vtk([actor_mesh(p, wireframe=True) 
           for s, p in standard_shape_level05surface_polydata_wrt_canonicalAtlasSpace_all_structures.iteritems()] \
            + [actor_sphere((0,0,0), radius=1)])

In [ ]:
# Show multiple iso-surfaces

standard_shape_level010surface_polydata_wrt_canonicalAtlasSpace_all_structures = \
DataManager.load_meshes_v2(atlas_spec, levels=.1, include_surround=True, 
                           structures=all_known_structures_sided)
#                            structures=all_known_structures_sided + all_known_structures_sided_with_surround_200um)

standard_shape_level090surface_polydata_wrt_canonicalAtlasSpace_all_structures = \
DataManager.load_meshes_v2(atlas_spec, levels=.9, include_surround=True, 
                           structures=all_known_structures_sided)
#                            structures=all_known_structures_sided + all_known_structures_sided_with_surround_200um)

launch_vtk([actor_mesh(p, wireframe=False, color=name_unsided_to_color_float[convert_to_unsided_label(s)], opacity=.2) 
           for s, p in standard_shape_level010surface_polydata_wrt_canonicalAtlasSpace_all_structures.iteritems()] \
           + [actor_mesh(p, wireframe=False, color=name_unsided_to_color_float[convert_to_unsided_label(s)], opacity=.1) 
           for s, p in standard_shape_level090surface_polydata_wrt_canonicalAtlasSpace_all_structures.iteritems()] \
            + [actor_sphere((0,0,0), radius=1)])

In [ ]:
# Load shell

stack_fixed = 'MD589'
shell_polydata_rel2fixedWholebrain = DataManager.load_mesh_v2(brain_spec={'name':stack_fixed, 
                                                                    'vol_type':'annotationAsScore',
                                                                   'resolution':'10.0um'}, 
                                                        structure='shell')

shell_polydata_rel2canonicalAtlasSpace = move_polydata(shell_polydata_rel2fixedWholebrain,
                                                       -canonical_center_wrt_fixed_um / convert_resolution_string_to_um(resolution='10.0um'))

shell_actor_rel2canon = actor_mesh(shell_polydata_rel2fixedWholebrain, (1,1,1), opacity=.15, 
                              wireframe=False, origin=-canonical_center_wrt_fixed_um / convert_resolution_string_to_um(resolution='10.0um'))

In [ ]:
save_data(shell_polydata_rel2canonicalAtlasSpace, 
          DataManager.get_mesh_filepath_v2(atlas_spec, structure='shell', level=None))

In [ ]:
launch_vtk([actor_mesh(p, wireframe=False, color=name_unsided_to_color_float[convert_to_unsided_label(s)], opacity=.2) 
           for s, p in standard_shape_level010surface_polydata_wrt_canonicalAtlasSpace_all_structures.iteritems()] \
           + [actor_mesh(p, wireframe=False, color=name_unsided_to_color_float[convert_to_unsided_label(s)], opacity=.1) 
           for s, p in standard_shape_level090surface_polydata_wrt_canonicalAtlasSpace_all_structures.iteritems()] \
            + [actor_sphere((0,0,0), radius=1)]\
#           +[shell_actor_rel2canon] \
           +[actor_mesh(shell_polydata_rel2canonicalAtlasSpace, (1,1,1), opacity=.15, 
                              wireframe=False)]
#           +[actor_volume(shell_vol_origin_dict_rel2canon['shell'][0].astype(np.float32), 
#                          what='probability', origin=shell_vol_origin_dict_rel2canon['shell'][1])]
          )

In [ ]:
# atlas_volume, structure_to_label, label_to_structure = \
# DataManager.load_original_volume_all_known_structures(stack=atlas_name, sided=True)

In [ ]:
vol, origin_rel2canon = structure_vol_origin_dicts_rel2canon['7N_L']

one_structure_volume_actor_rel2canon = actor_volume(vol.astype(np.float32), what='score', 
                                              origin=origin_rel2canon,
                                                    auxdata=0.8*(vol>0.1).astype(np.float32),
                                              c=np.array(name_unsided_to_color['7N'])/255.)
#                                             c=np.array((1,0,0)))

# Render

In [ ]:
launch_vtk([] \
    + structure_mesh_actors_rel2canon \
#     + [one_structure_volume_actor_rel2canon] \
#     + [shell_volume_actor_rel2canon] \
#     + structure_mesh_surround_actors_rel2canon \
    + [shell_actor_rel2canon] \
    + [actor_sphere((0,0,0), radius=1)], 
           init_angle='sagittal', 
    background_color=(1,1,1),
depth_peeling=True)